In [ ]:
# 1. 필수 임포트
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, MessagesState
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

# 2. 메시지 상태 정의
class AgentState(MessagesState):
    pass

# 3. LLM 설정
llm = ChatOpenAI(model="gpt-4o")

# ✅ 4. 메시지 처리 래퍼: 메시지 리스트 -> 메시지 리스트
def agent_node(state: AgentState) -> AgentState:
    messages = state["messages"]
    response = llm.invoke(messages)
    return {"messages": messages + [response]}

# 5. 도구 정의
@tool
def search_cafe_menu(query: str) -> str:
    """카페 메뉴 정보를 검색합니다."""
    if "아메리카노" in query:
        return "아메리카노는 4,500원이며, 아이스 아메리카노는 시원한 물과 얼음을 추가한 변형입니다."
    elif "라떼" in query:
        return "카페라떼는 우유 기반, 바닐라라떼는 바닐라 시럽 추가, 녹차라떼는 녹차 파우더를 사용합니다."
    elif "티라미수" in query:
        return "티라미수는 에스프레소를 적신 시트와 마스카포네 크림이 층을 이루는 디저트입니다."
    else:
        return "요청하신 메뉴에 대한 정보가 없습니다."

tool_node = {"search_cafe_menu": search_cafe_menu}

# 6. 그래프 구성
builder = StateGraph(AgentState)
builder.add_node("agent", agent_node)
builder.add_node("tools", tool_node)
builder.add_conditional_edges("agent", tools_condition)
builder.add_edge("tools", "agent")
builder.set_entry_point("agent")
builder.set_finish_point("agent")

graph = builder.compile()

# 7. 테스트 함수
def test_agent(query: str):
    print(f"\n🧑 USER: {query}")
    response = graph.invoke({"messages": [HumanMessage(content=query)]})
    print("🤖 AI 응답:")
    for msg in response["messages"]:
        print(f"- {msg.content}")

# 8. 실행
test_agent("아메리카노와 아이스 아메리카노의 차이점과 가격을 알려주세요.")
test_agent("라떼 종류에는 어떤 메뉴들이 있고 각각의 특징은 무엇인가요?")
test_agent("디저트 메뉴 중에서 티라미수에 대해 자세히 설명해주세요.")
